In [40]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, InputLayer, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import tensorflow as tf
from PIL import Image
import json
from pathlib import Path
from keras.models import model_from_json
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9

tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [41]:
def preprocess(image):
    #resized_image = tf.image.resize(image,[224,224])
    final_image=keras.applications.resnet50.preprocess_input(image)
    return final_image

In [42]:
train_datagen = ImageDataGenerator(validation_split = 0.2,preprocessing_function=preprocess)

In [43]:
train_generator = train_datagen.flow_from_directory(
        r'D:\Downloads\Copy of shopee-product-detection-dataset\train_10\train_10',
        subset = 'training',# this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=16,
        color_mode="rgb",
        class_mode="categorical",
        shuffle=True,
        seed=42) 


val_generator = train_datagen.flow_from_directory(
        r'D:\Downloads\Copy of shopee-product-detection-dataset\train_10\train_10',
        subset = 'validation',# this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=16,
        color_mode="rgb",
        class_mode="categorical",
        shuffle=True,
        seed=42) 


Found 18575 images belonging to 10 classes.
Found 4638 images belonging to 10 classes.


In [55]:
# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model_1 = model_from_json(model_structure)

# Re-load the model's trained weights
model_1.load_weights("model_resnet.h5")

In [56]:
model_1.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [57]:

model = Sequential()
model.add(Conv2D(3, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(3, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [58]:
for layer in model_1.layers[1:]:
    model.add(layer)

In [59]:
for layer in model.layers:
    layer.trainable=True

In [60]:
optimizer=keras.optimizers.SGD(lr=0.2,momentum=0.9,decay=0.01)
model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])

In [61]:
history=model.fit(train_generator,epochs=5,validation_data=val_generator)

RuntimeError: You must compile your model before using it.